In [1]:
import time; 

#io
import os
import json

#data
import numpy as np
import pandas as pd



##Hit level

In [2]:
types = ["ECAL","HCAL","IT","OT","TT","VELOR1","VELOR2","VELOPHI1","VELOPHI2","MUON","MUON2"]
type_code = {c:i for i,c in enumerate(types)}

def collect_hits(evt,types_dict = type_code):
    """collect hits (np.array) from a json event dictionary
    output: np.array([x,y,z,type_code]) of hits"""

    arrays = []
    
    #calorimeters:0
    for cal in "ECAL","HCAL":
        XYZ = [hit[1:4]+[types_dict[cal]] for hit in evt[cal]]
        arrays.append(np.array(XYZ))
    #triggers: 1
    for trigger in "TT","OT","IT":
        XYZ1 = [hit[0:3]+[types_dict[trigger]] for hit in evt[trigger]]
        arrays.append(np.array(XYZ1))
    #muon detector (x:2,x+dx:3)
    if "MUON" in evt:
        XYZmu = np.array([hit[::2]+[types_dict["MUON"]] for hit in evt["MUON"]])
        arrays.append(XYZmu)

        deltaXYZmu = np.array([hit[1::2]+[1] for hit in evt["MUON"]])
        XYZmu2 = XYZmu+deltaXYZmu
        XYZmu2[:,-1] = types_dict["MUON2"]
        arrays.append(XYZmu2)
    #velo detector (velor:4, velophi:5)
    for velo in "VELOR","VELOPHI":
        XYZ1 = [hit[:3]+[types_dict[velo+"1"]] for hit in evt[velo]]
        arrays.append(np.array(XYZ1))

        XYZ2 = [hit[:3]+[types_dict[velo+"2"]] for hit in evt[velo]]
        arrays.append(np.array(XYZ2))
    #compose a single array
    arrays = filter(lambda a:len(a)!=0,arrays)
    points = np.vstack(arrays)
    return points

In [3]:
import pandas
def extract_hits(evt):

    if len(evt["ECAL"]):
        hits = collect_hits(evt,type_code)
        df = pandas.DataFrame(hits,columns = ['X','Y','Z','source'])
        df["source"] = np.array(types)[df["source"].values.astype(int)]
        return df
    return False

In [4]:
def extract_tracks(evt):
    particle_dicts = evt["PARTICLES"]
    track_rows = []
    for pdict in particle_dicts:
        pdict = dict(pdict) #shallowcopy
        track = pdict["track"]
        for i,(x,y,z) in enumerate(track):
            pdict['x'+str(i)] = x
            pdict['y'+str(i)] = y
            pdict['z'+str(i)] = z
        del pdict["track"]
        track_rows.append(pdict)
    
    df = pandas.DataFrame(track_rows)
    return df        
            

In [5]:
def preprocess(json_folder,outdir_tracks,outdir_hits,n_shards=1,shard_i=0,omit_tracks=False):
    itr=0
    names = filter(lambda fname: fname.endswith(".json"),os.listdir(json_folder))

    names = names[shard_i::n_shards]
    
    try: os.mkdir(outdir_hits)
    except:pass 
    try: os.mkdir(outdir_tracks)
    except:pass 
    
    paths = np.array(map(lambda fname: os.path.join(json_folder,fname),names))
    #shuffle
    paths = paths[np.argsort(np.random.random(size = len(paths)))]

    for name,path in zip(names,paths):
        eventid = name.split(".")[0]#runnumber_eventnumber
        try:
            evt = json.load(open(path))
        except:
            continue
        hit_df = extract_hits(evt)

        if hit_df is not False:

            hit_df.to_csv(os.path.join(outdir_hits,eventid+".hits.csv"))
            
            if not omit_tracks:
                track_df = extract_tracks(evt)
                track_df.to_csv(os.path.join(outdir_tracks,eventid+".tracks.csv"))
            itr+=1

            if itr%1000 ==0:
                print itr, "events"

In [6]:
"""
rm -rf /mnt/KSfinder/mc_bg/bg_jsons_*
tar -xvf /root/bg_down.tar.gz -C /mnt/KSfinder/mc_bg/
mv /mnt/KSfinder/mc_bg/bg_jsons_all/ /mnt/KSfinder/mc_bg/bg_jsons_down/
tar -xvf /root/bg_up.tar.gz -C /mnt/KSfinder/mc_bg/
tar -xvf /root/sig_jsns.tar.gz -C /mnt/KSfinder/mc_sig/
"""

'\nrm -rf /mnt/KSfinder/mc_bg/bg_jsons_*\ntar -xvf /root/bg_down.tar.gz -C /mnt/KSfinder/mc_bg/\nmv /mnt/KSfinder/mc_bg/bg_jsons_all/ /mnt/KSfinder/mc_bg/bg_jsons_down/\ntar -xvf /root/bg_up.tar.gz -C /mnt/KSfinder/mc_bg/\ntar -xvf /root/sig_jsns.tar.gz -C /mnt/KSfinder/mc_sig/\n'

In [ ]:
from sklearn.externals import joblib

for fname in os.listdir("../data/jsons/"):
    print "processing",fname
    n_shards = 1
    tasks =[
        joblib.delayed(preprocess)(
            json_folder= "../data/jsons/%s"%(fname),
            outdir_hits = "../data/%s"%(fname),
            outdir_tracks = "../data/%s"%(fname),
            n_shards= n_shards,
            shard_i=i
        )
        for i in range(n_shards)
    ]

    _=joblib.Parallel(n_jobs=1)(tasks)


processing bg_down
100 events
200 events
300 events
400 events
500 events
600 events
700 events
800 events
900 events
1000 events
1100 events
1200 events
1300 events
1400 events
1500 events
1600 events
1700 events
1800 events
1900 events
2000 events
2100 events
2200 events
2300 events
2400 events
2500 events
2600 events
2700 events
2800 events
2900 events
3000 events
3100 events
3200 events
3300 events
3400 events
3500 events
3600 events
3700 events
3800 events
3900 events
4000 events
4100 events
4200 events
4300 events
4400 events
4500 events
4600 events
4700 events
4800 events
4900 events
5000 events
5100 events
5200 events
5300 events
5400 events
5500 events
5600 events
5700 events
5800 events
5900 events
6000 events
6100 events
6200 events
6300 events
6400 events
6500 events
6600 events
6700 events
6800 events
6900 events
7000 events
7100 events
7200 events
7300 events
7400 events
7500 events
7600 events
7700 events
7800 events
7900 events
8000 events
8100 events
8200 events
8300 e

In [12]:
!ls ../data/ -lh

total 20M
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 2,2M апр.   3 00:19 11104101_down
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 2,3M апр.   2 23:44 11104101_up
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 2,2M апр.   2 23:34 11154100_down
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 2,2M апр.   3 00:29 11154100_up
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 4,0K апр.   1 13:13 4retinas
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 4,0K апр.   2 22:48 archives
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 2,6M апр.   2 23:24 bg_down
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 2,0M апр.   2 23:52 bg_up
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 4,0K апр.   2 23:22 decays
drwxr-xr-x 9 jheuristic dpt_yandex_factory_academ 4,0K апр.   2 23:05 jsons
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 2,5M апр.   2 23:17 sig56
drwxr-xr-x 2 jheuristic dpt_yandex_factory_academ 3,3M апр.   3 00:09 sig_56
